In [1]:
import warnings
warnings.filterwarnings('ignore');

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
# Use the ChromeDriverManager to automatically download and set up the appropriate ChromeDriver
driver = webdriver.Chrome()
url='https://www.flipkart.com/'
driver=webdriver.Chrome(executable_path=path)
driver.get(url)

In [15]:
# order section of flipkart
driver.refresh()
try:
   driver.get('https://www.flipkart.com/account/orders?link=home_orders')
except Exception as e:
    print(e)

In [16]:
#: simple function to scroll the html body
def scroll():
    def scroll_window():
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    while(True):
        last_height=driver.execute_script("return document.body.scrollHeight")
        scroll_window()
        time.sleep(2)
        new_height=driver.execute_script("return document.body.scrollHeight")
        if(last_height==new_height):
            break

In [ ]:
#: scroll untill we reach the end of the page
driver.refresh()
flag=True
scroll()
while flag:
    try:
        _more_btn=driver.find_element(By.XPATH,"//button[@class='_2KpZ6l _2jekE6']")
        if _more_btn:
            _more_btn.click()
            time.sleep(3)
            scroll()
        else:
            flag=False
            break
            print('more order button not found now')
    except Exception as e:
            time.sleep(2)
            last_btn=driver.find_element(By.XPATH,"//div[@class='_2jekE6']")
            if last_btn.is_displayed() and 'No More Results' in last_btn.text:
                print('no more orders we are at the end of the page')
                break
    


In [26]:
#" here we are extracting the html source of the orders list page-(this page contains list of all the orders)
source=driver.page_source
from bs4 import BeautifulSoup
soup=BeautifulSoup(source)

In [27]:
#: extract the list of all the orders
_items=soup.find('div',class_='column _2Uo9IJ')

In [28]:
#: check the number of orders
order_divs=_items.find_all('div',class_='_35va4c')
len(order_divs)

222

In [151]:
order_links=[]
order_titles=[]
order_status=[]
order_ids=[]
order_prices=[]
order_type=[]
date=[]
for i,order in enumerate(order_divs):
    #: extracting the order link, and order id from the link
    #: ***********************************************************************************************\
    
    try:
        link="https://www.flipkart.com"+order.find('a',class_='_3dshlq').get('href')
        order_links.append(link) 
        
    #:************************************************************************************************
     #: extracting the order id from the links
        
        _id=link.split("=")[1].split("&")[0]
        order_ids.append(_id)
    except:
        order_links.append(None)
        order_ids.append(None)
   
     #:***************************************************************************************************************
     #: now extracting the order_status of each order 
        
    try:
        _status=order.find('span',class_='AO0UbU').text
        if 'Failed' in _status:
            date.append(None)
            order_status.append('Failed')
        elif 'Not Placed' in _status:
            date.append(None)
            order_status.append('Not Placed')
        elif 'Cancelled' in _status:
            date.append(None)
            order_status.append('Cancelled')
        elif 'Refund' in _status:
            date.append(None)
            order_status.append('Refunded')
        elif 'Replacement' in _status:
            date.append(None)
            order_status.append('Replacement')
        else:
            dt=_status.split("on ")[-1]
            date.append(dt)
            order_status.append('Delivered')
        
    except Exception as e:
        _status.append(None)
    
    #:***************************************************************************************************************
    #:extracting the price from the orders:
    
    try:
        cost=order.find('div',class_='col-2-12 _1_SN5_').text.replace("₹","")
        order_prices.append(cost)
    except:
        print('cost not found')
        order_prices.append(None)
        
    #:***************************************************************************************************************
    #:extracting the title from the orders
    
    try:
        title=order.find('span',class_='row _1kkfO3 BqOr_g').text
        order_titles.append(title)
    except:
        order_titles.append(None)

In [152]:
print(len(order_status))
print(len(order_ids))
print(len(order_links))
print(len(order_titles))
print(len(date))

222
222
222
222
222


In [153]:
#: taking the order id, title , price, status and creating a dataframe from them
all_orders=pd.DataFrame({'Order_Ids':order_ids,'Order_Title':order_titles,'Order_Source':order_links,'Order_Status':order_status,'price':order_prices,'Date':date})
all_orders.head()

,Order_Ids,Order_Title,Order_Source,Order_Status,price,Date
0,OD329454879035222300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Delivered,29,Thu Oct 19
1,OD329454872649229300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,29,None
2,OD329454866261449300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,18 +1,None
3,OD329454865341232300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,19,None
4,OD329441200245284300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Delivered,19,Tue Oct 17


#### Cleaning the all orders dataframe

In [154]:
all_orders.head(6)

,Order_Ids,Order_Title,Order_Source,Order_Status,price,Date
0,OD329454879035222300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Delivered,29,Thu Oct 19
1,OD329454872649229300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,29,None
2,OD329454866261449300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,18 +1,None
3,OD329454865341232300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,19,None
4,OD329441200245284300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Delivered,19,Tue Oct 17
5,OD329376608982457100,HARPA Women Fit and Flare Pink Dress,https://www.flipkart.com/order_details?order_i...,Delivered,494,Oct 14


In [155]:
all_orders.isnull().sum()

Order_Ids         2
Order_Title       2
Order_Source      2
Order_Status      0
price             0
Date            129
dtype: int64

In [156]:
#: cleaning the price column:
def cleaning_price(price):
    price=price.replace(",","")
    if "+" in price:
        prices=price.split("+")
        prices=[int(i) for i in prices]
        return sum(prices)
    else:
        return int(price)
all_orders['price']=all_orders['price'].apply(cleaning_price)

In [157]:
#: cleaning the order ids:
all_orders['Order_Ids']=all_orders['Order_Ids'].apply(lambda x: (x.replace("OD","")) if x!=None else x)

In [158]:
#: filtering out the recharges
all_orders['Order_Type']=all_orders['Order_Title'].apply(lambda x: 'Mobile Recharge' if x!=None and 'Recharge' in x else 'Shopping')

In [159]:
all_orders.head()

,Order_Ids,Order_Title,Order_Source,Order_Status,price,Date,Order_Type
0,329454879035222300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Delivered,29,Thu Oct 19,Mobile Recharge
1,329454872649229300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,29,None,Mobile Recharge
2,329454866261449300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,19,None,Mobile Recharge
3,329454865341232300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,19,None,Mobile Recharge
4,329441200245284300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Delivered,19,Tue Oct 17,Mobile Recharge


In [160]:
all_orders['Order_Type'].value_counts()

Order_Type
Shopping           211
Mobile Recharge     11
Name: count, dtype: int64

In [161]:
#: filter out the number of the delivered and refunded orders:
print('shape-',all_orders.query('Order_Status=="Refunded" | Order_Status=="Delivered"').shape)
print("*"*50)
print('Value Counts:\n' ,all_orders['Order_Status'].value_counts())
print("*"*50)
(all_orders.query('Order_Status=="Refunded" | Order_Status=="Delivered"'))

shape- (172, 7)
**************************************************
Value Counts:
 Order_Status
Delivered      93
Refunded       79
Cancelled      39
Failed          8
Not Placed      2
Replacement     1
Name: count, dtype: int64
**************************************************


,Order_Ids,Order_Title,Order_Source,Order_Status,price,Date,Order_Type
0,329454879035222300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Delivered,29,Thu Oct 19,Mobile Recharge
4,329441200245284300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Delivered,19,Tue Oct 17,Mobile Recharge
5,329376608982457100,HARPA Women Fit and Flare Pink Dress,https://www.flipkart.com/order_details?order_i...,Delivered,494,Oct 14,Shopping
6,329347858690695100,ZEBRONICS ZEB-TRANSFORMER-M Wired Optica...,https://www.flipkart.com/order_details?order_i...,Delivered,359,Oct 11,Shopping
7,329347787410570100,ASUS TUF Gaming K3 - RA05 / 100% Anti-gh...,https://www.flipkart.com/order_details?order_i...,Delivered,2949,Oct 10,Shopping
...,...,...,...,...,...,...,...
210,119962023262142000,LooMantha Fridge Mat,https://www.flipkart.com/order_details?order_i...,Delivered,179,"Oct 19, 2020",Shopping
211,119962007071261000,Camlin 2 in 1 Geometry and Pencil Box Se...,https://www.flipkart.com/order_details?order_i...,Delivered,149,"Oct 20, 2020",Shopping
216,219796133033930000,Metronaut Typography Women Henley Neck W...,https://www.flipkart.com/order_details?order_i...,Refunded,262,None,Shopping
218,119583500676962000,YBT FIXER WITH FACE PRIMER,https://www.flipkart.com/order_details?order_i...,Delivered,212,"Sep 06, 2020",Shopping


In [162]:
all_orders.head()

,Order_Ids,Order_Title,Order_Source,Order_Status,price,Date,Order_Type
0,329454879035222300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Delivered,29,Thu Oct 19,Mobile Recharge
1,329454872649229300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,29,None,Mobile Recharge
2,329454866261449300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,19,None,Mobile Recharge
3,329454865341232300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,19,None,Mobile Recharge
4,329441200245284300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Delivered,19,Tue Oct 17,Mobile Recharge


In [18]:
# all_orders.to_csv("All Orders List.csv",index=False)
all_orders=pd.read_csv("All Orders List.csv")

In [163]:
all_orders.dropna(subset=['Order_Ids'],inplace=True)

In [27]:
all_orders.dropna(subset=['Order_Source'],inplace=True)

#### Extracting more details of delivered and refunded orders by visiting each order urls

In [ ]:
#: extracting the data of the deliverd orders:
#: we need phone number name, pincode, order/refund/date or any sort of date/seller name
import time
data=all_orders.query('(Order_Type == "Shopping") & ((Order_Status == "Delivered" | Order_Status == "Refunded") | (Order_Status == "Cancelled"))')
customer_name=[]
seller_name=[]
phone_number=[]
address=[]
_dates=[]
_sources=[]
for idx, value in data.iterrows():
  
    #: check if this is refunded order
  if value['Order_Status']=='Refunded' or value['Order_Status']=='Cancelled':
    try:
        _date=(driver.find_element(By.XPATH,"//div[@class='_3e6B3t']").text)
        _dates.append(_date)
    except:
        _dates.append(_date)
  link=value['Order_Source']
  if not link is None :
    driver.get(link)
    time.sleep(3)
     #:***************************************************************************************************************
    #: extracting the customer names
    try:
        cus_name=driver.find_element(By.XPATH,'//*[@id="container"]/div/div[3]/div[2]/div/div[4]/div[1]/div/div[2]/div[1]/div').text
        customer_name.append(cus_name)
    except:
        customer_name.append(None)
    #:***************************************************************************************************************
    #: extracting the seller name
    try:
       seller=driver.find_element(By.XPATH,'//*[@id="container"]/div/div[3]/div[2]/div/div[6]/div/div[1]/div[1]/div/div[2]/div[2]/span[2]').text
       seller_name.append(seller)
    except:
        try:
            seller=driver.find_elements(By.XPATH,"//span[@class='_5CyX2n']")[-1].text
            seller_name.append(seller)
        except:
            seller_name.append(None)
     #:***************************************************************************************************************
    #: extracting the address
    try:
        add=driver.find_element(By.XPATH,"//div[@class='_10YZ6u']").text
        address.append(add)
    except:
        address.append(None)
     #:***************************************************************************************************************
    #: extracting the mobile number
    try:
        phn_no=driver.find_element(By.XPATH,"//div[@class='_23g6G- _2rcD-x']").text
        phone_number.append(phn_no)
    except:
        phone_number.append(None)
    #:extracting the url of the product
#     **************************************************************************************************************************
    try:
        product_link=driver.find_element(By.XPATH,"//a[@class='_2KpZ6l row _3hTjfv']").get_attribute('href')
        _sources.append(product_link)
    except:
        print('Link not Found')
        _sources.append(None)
print('successful')

In [45]:
print(len(phone_number))
print(len(address))
print(len(seller_name))
print(len(customer_name))
print(len(_sources))
print(data.shape)

206
206
206
206
206
(206, 7)


In [46]:
#: adding the extracted data to the all orders dataframe
all_orders.head(3)
all_orders['Customer_Name']=None
all_orders['Phone_number']=None
all_orders['address']=None
all_orders['seller_name']=None
all_orders['Product_Link']=None

In [47]:
all_orders.head(3)

,Order_Ids,Order_Title,Order_Source,Order_Status,price,Date,Order_Type,Customer_Name,Phone_number,address,seller_name,Product_Link
0,OD329454879035222300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Delivered,29,Thu Oct 19,Mobile Recharge,None,None,None,None,None
1,OD329454872649229300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,29,NaN,Mobile Recharge,None,None,None,None,None
2,OD329454866261449300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,19,NaN,Mobile Recharge,None,None,None,None,None


In [48]:
#: saving the values
query =(all_orders['Order_Type'] == "Shopping") & ((all_orders['Order_Status'] == "Delivered") | (all_orders['Order_Status'] == "Refunded") | (all_orders['Order_Status'] == "Cancelled"))
all_orders.loc[query,'Customer_Name']=customer_name
all_orders.loc[query,'Phone_number']=phone_number
all_orders.loc[query,'address']=address
all_orders.loc[query,'seller_name']=seller_name
all_orders.loc[query,'Product_Link']=_sources
all_orders.loc[5:15]

,Order_Ids,Order_Title,Order_Source,Order_Status,price,Date,Order_Type,Customer_Name,Phone_number,address,seller_name,Product_Link
5,OD329376608982457100,HARPA Women Fit and Flare Pink Dress,https://www.flipkart.com/order_details?order_i...,Delivered,494,Oct 14,Shopping,Ankit Srivastava,Phone number9838656023,"Shiv Colony, Chhata , Govardhan Chauraha Near ...",HSAtlastradeFashion,https://www.flipkart.com/harpa-women-fit-flare...
6,OD329347858690695100,ZEBRONICS ZEB-TRANSFORMER-M Wired Optica...,https://www.flipkart.com/order_details?order_i...,Delivered,359,Oct 11,Shopping,Ankit Srivastava,Phone number9838656023,"Shiv Colony, Chhata , Govardhan Chauraha Near ...",ECOMDIGITALRetail,https://www.flipkart.com/zebronics-zeb-transfo...
7,OD329347787410570100,ASUS TUF Gaming K3 - RA05 / 100% Anti-gh...,https://www.flipkart.com/order_details?order_i...,Delivered,2949,Oct 10,Shopping,Ankit Srivastava,Phone number9838656023,"Shiv Colony, Chhata , Govardhan Chauraha Near ...",SVPeripherals,https://www.flipkart.com/asus-tuf-gaming-k3-ra...
8,OD329347713421991100,ASUS TUF Gaming K3 - RA05 / 100% Anti-gh...,https://www.flipkart.com/order_details?order_i...,Not Placed,2949,NaN,Shopping,None,None,None,None,None
9,OD329347692199775100,ASUS TUF Gaming K3 - RA05 / 100% Anti-gh...,https://www.flipkart.com/order_details?order_i...,Not Placed,2949,NaN,Shopping,None,None,None,None,None
10,OD329325070133090300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,149,NaN,Mobile Recharge,None,None,None,None,None
11,OD329325055389892300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,149,NaN,Mobile Recharge,None,None,None,None,None
12,OD329298846222439100,THE WOOD WHITE INDIA Soft 2. 16 x 24 Inc...,https://www.flipkart.com/order_details?order_i...,Delivered,395,Oct 05,Shopping,Ankit Srivastava,Phone number9838656023,"Shiv Colony, Chhata , Govardhan Chauraha Near ...",theWoodwhite,https://www.flipkart.com/wood-white-india-soft...
13,OD329281807056729300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Delivered,99,Sep 29,Mobile Recharge,None,None,None,None,None
14,OD329281798812030300,Mobile Prepaid Recharge,https://www.flipkart.com/order_details?order_i...,Failed,99,NaN,Mobile Recharge,None,None,None,None,None


In [88]:
all_orders['Product_Link'][0]

'https://www.flipkart.com/harpa-women-fit-flare-pink-dress/p/itmf3vb5aquuebh8?pid=DREESXZBYZJV6AJT&lid=LSTDREESXZBYZJV6AJTI2NMKX'

In [49]:
#: saving the dates in the all orders dataframe
query=( (all_orders.Order_Status=='Refunded') | (all_orders.Order_Status=='Cancelled') )
all_orders.loc[query,'Date']=_dates
all_orders.loc[query]

,Order_Ids,Order_Title,Order_Source,Order_Status,price,Date,Order_Type,Customer_Name,Phone_number,address,seller_name,Product_Link
18,OD329121785871884100,USHA EI 2802 LT 1000 W Dry Iron,https://www.flipkart.com/order_details?order_i...,Refunded,499,"Wed, 13th Sep",Shopping,Ankit Srivastava,Phone number9838656023,"Shiv Colony, Chhata , Govardhan Chauraha Near ...",A1SHOPPINGMART,https://www.flipkart.com/usha-ei-2802-lt-1000-...
21,OD328952734694021100,Thecrazymonk Graphic Print Men Round Nec...,https://www.flipkart.com/order_details?order_i...,Refunded,699,"Thu, 7th Sep",Shopping,Renu Srivastav,"Phone number\n9129707398, 9838656023","2013,awas vikas colony BharatPuri Bharatpuri, ...",Crazymonkin,https://www.flipkart.com/thecrazymonk-graphic-...
25,OD328662721842005100,Sansu Olive Cider Vinegar Premium (Jaitu...,https://www.flipkart.com/order_details?order_i...,Cancelled,354,"Thu, 20th Jul",Shopping,None,"Phone number\n9129707398, 9452137398","Pehar bazar, uttraula , balrampur Near Pehar p...",KRSFOODS,https://www.flipkart.com/sansu-olive-cider-vin...
29,OD328083566574607100,DHARITRI SAREE Self Design Kanjivaram Pu...,https://www.flipkart.com/order_details?order_i...,Refunded,870,"Mon, 22nd May",Shopping,Renu Srivastava,"Phone number\n9129707398, 9838656023","Road No 8, Semra Colony, Ranijot At Umesh Sriv...",DharitriSaree,https://www.flipkart.com/dharitri-saree-self-d...
34,OD327788654157330100,RED TAPE Sneakers For Women,https://www.flipkart.com/order_details?order_i...,Refunded,959,"Mon, 10th Apr",Shopping,Ankit Srivastava,Phone number9838656023,"Shiv Colony, Chhata , Govardhan Chauraha Near ...",REDTAPELIMITED,https://www.flipkart.com/red-tape-sneakers-wom...
...,...,...,...,...,...,...,...,...,...,...,...,...
215,OD119960242580668000,KANUSHI Refrigerator Cover,https://www.flipkart.com/order_details?order_i...,Cancelled,172,"Thu, 15th Oct",Shopping,Ankit Srivastava,Phone number9838656023,"240v Pehar bazar , Near pehar mata mandir, Utr...",KanushiIndustries,https://www.flipkart.com/kanushi-refrigerator-...
216,OD219796133033930000,Metronaut Typography Women Henley Neck W...,https://www.flipkart.com/order_details?order_i...,Refunded,262,"Thu, 15th Oct",Shopping,Ankit Shrivastav,"Phone number\n9838656023, 7379986114","240v Pehar mata mandir, Utraula - 271604, Utta...",maniaclife,https://www.flipkart.com/metronaut-typography-...
217,OD219795772647677000,Metronaut Printed Women Round Neck White...,https://www.flipkart.com/order_details?order_i...,Cancelled,263,"Wed, 30th Sep",Shopping,Ankit Shrivastav,Phone number9838656023,"Ankit shrivastav Pehar Mata Mandir, Utraula Su...",maniaclife,https://www.flipkart.com/metronaut-womens-roun...
220,OD115563353378372000,Khadi Natural Neem andTulsi Face Wash,https://www.flipkart.com/order_details?order_i...,Cancelled,175,"Sun, 24th Nov",Shopping,None,"Phone number\n9838656602, 8858796805","2013 Awas vikas colony ,bharatpuri , near awas...",KHADIONLINEMKT,https://www.flipkart.com/khadi-natural-neem-an...


In [50]:
#: Data Cleaning and EDA
all_orders['Order_Status'].value_counts()

Order_Status
Delivered      91
Refunded       79
Cancelled      39
Failed          8
Not Placed      2
Replacement     1
Name: count, dtype: int64

In [51]:
all_orders=all_orders.loc[~ (all_orders.Order_Type=='Mobile Recharge')].reset_index(drop=True)
all_orders.head()

,Order_Ids,Order_Title,Order_Source,Order_Status,price,Date,Order_Type,Customer_Name,Phone_number,address,seller_name,Product_Link
0,OD329376608982457100,HARPA Women Fit and Flare Pink Dress,https://www.flipkart.com/order_details?order_i...,Delivered,494,Oct 14,Shopping,Ankit Srivastava,Phone number9838656023,"Shiv Colony, Chhata , Govardhan Chauraha Near ...",HSAtlastradeFashion,https://www.flipkart.com/harpa-women-fit-flare...
1,OD329347858690695100,ZEBRONICS ZEB-TRANSFORMER-M Wired Optica...,https://www.flipkart.com/order_details?order_i...,Delivered,359,Oct 11,Shopping,Ankit Srivastava,Phone number9838656023,"Shiv Colony, Chhata , Govardhan Chauraha Near ...",ECOMDIGITALRetail,https://www.flipkart.com/zebronics-zeb-transfo...
2,OD329347787410570100,ASUS TUF Gaming K3 - RA05 / 100% Anti-gh...,https://www.flipkart.com/order_details?order_i...,Delivered,2949,Oct 10,Shopping,Ankit Srivastava,Phone number9838656023,"Shiv Colony, Chhata , Govardhan Chauraha Near ...",SVPeripherals,https://www.flipkart.com/asus-tuf-gaming-k3-ra...
3,OD329347713421991100,ASUS TUF Gaming K3 - RA05 / 100% Anti-gh...,https://www.flipkart.com/order_details?order_i...,Not Placed,2949,NaN,Shopping,None,None,None,None,None
4,OD329347692199775100,ASUS TUF Gaming K3 - RA05 / 100% Anti-gh...,https://www.flipkart.com/order_details?order_i...,Not Placed,2949,NaN,Shopping,None,None,None,None,None


In [106]:
#: creating a new df which only contains the delivered returned and cancelled records, we dont need failed or order not placed orders
query=((all_orders['Order_Status'] == "Delivered") | (all_orders['Order_Status'] == "Refunded") | (all_orders['Order_Status'] == "Cancelled"))
Orders_List=all_orders.loc[query]

In [107]:
Orders_List.isnull().sum()
Orders_List=Orders_List.loc[~(Orders_List.address.isnull())]

#### : cleaning the order_list DataFrame

##### : cleaning the phone number columns

In [108]:
Orders_List['Phone_number']=Orders_List['Phone_number'].apply(lambda x: int(x.replace("\n","").replace(",","").replace("Phone number","").split(" ")[0]))

##### : cleanining the address and extarting the pincode and state/ut from it

In [109]:
Orders_List['pincode']=Orders_List['address'].apply(lambda x: int(x.split("-")[-1].split(",")[0]))
Orders_List['state/ut']=Orders_List['address'].apply(lambda x: x.split("-")[-1].split(",")[-1])

In [110]:
#: so we no longer need the address columns so lets drop it
Orders_List.drop(columns=['address'],inplace=True)

##### Extracting the Product company from the title given

In [111]:
Orders_List['Company']=Orders_List['Order_Title'].apply(lambda x: x.split(" ")[0])

In [112]:
Orders_List.columns

Index(['Order_Ids', 'Order_Title', 'Order_Source', 'Order_Status', 'price',
       'Date', 'Order_Type', 'Customer_Name', 'Phone_number', 'seller_name',
       'Product_Link', 'pincode', 'state/ut', 'Company'],
      dtype='object')

In [113]:
Orders_List=Orders_List[['Order_Ids','Customer_Name','Order_Title',
                        'Company','Order_Status','price','Date','Order_Type','Phone_number','seller_name','pincode',
                        'state/ut','Order_Source','Product_Link']]

In [114]:
Orders_List['Customer_Name'].fillna('Ankit Srivastava',inplace=True)

In [ ]:
#: we dont have the category of the products so we will visit the product link and extract the product categories
main_category=[]
sub_category=[]
prices=[]
for idx, value in Orders_List.iterrows():
    link=value['Product_Link']
    if not link is None:
        driver.get(link)
        time.sleep(1)
        #: extracting the main and sub category
        try:
            cats=driver.find_elements(By.XPATH,"//a[@class='_2whKao']")
            main_category.append(cats[1].text)
            sub_category.append(cats[2].text)
        except:
            main_category.append(None)
            sub_category.append(None)
        #: extractiong the new price of the product
        try:
            price=driver.find_element(By.XPATH,"//div[@class='_30jeq3 _16Jk6d']").text
            prices.append(price)
        except:
            prices.append(None)
print('success')

In [120]:
print(len(main_category))
print(len(sub_category))
print(len(prices))
print(Orders_List.shape)

205
205
205
(205, 14)


In [122]:
#: saving the extracted data into the dataframe
Orders_List['Main_category']=main_category
Orders_List['Sub_category']=sub_category
Orders_List['Current_Price']=prices

In [127]:
Orders_List=Orders_List[['Order_Ids','Customer_Name','Order_Title','Main_category','Sub_category','Company','Order_Status','price',
                         'Current_Price','Date','Order_Type','Phone_number','seller_name','pincode','state/ut','Order_Source','Product_Link']]

In [132]:
# Saving the cleaned data in csv
Orders_List.to_csv("Flipkart Order List.csv",index=False)